<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/models/Matthias_category_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import cluster
import random
import time
from io import BytesIO
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
! pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-ldt5v408
  Running command git clone -q https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-ldt5v408
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
     |████████████████████████████████| 11.7 MB 3.8 MB/s 
     |████████████████████████████████| 17.1 MB 12 kB/s 
     |████████████████████████████████| 11.3 MB 67.2 MB/s 
     |████████████████████████████████| 4.5 MB 28.1 MB/s 
     |████████████████████████████████| 61 kB 115 kB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 31.2 MB 1.3 MB/s 
     |███████████████████

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip '/content/drive/MyDrive/Colab Notebooks/Data Moda/images.zip' 

In [2]:
PATH = '/content/images'

In [3]:
from preproc.preprocess import AttributePreproc, SectionPreproc, CategoryPreproc

In [4]:
prep = CategoryPreproc(PATH, (224,224), 0.2)

In [5]:
X_train, X_test, y_train, y_test, names  = prep.run()

Augmenting category 'Tees'...
Augmenting category 'Dresses'...
Augmenting category 'Blouses'...
Augmenting category 'Sweaters'...
Augmenting category 'Rompers'...
Augmenting category 'Shorts'...
Augmenting category 'Pants'...
Augmenting category 'Jackets'...
Augmenting category 'Cardigans'...
Augmenting category 'Skirts'...
Augmenting category 'Joggers'...
Augmenting category 'Graphic_Tees'...
Augmenting category 'Shirts'...
Augmenting category 'Baggy_Pants'...
Augmenting category 'Suiting'...
Done!


# Model application

In [6]:
## Orginal Class code

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Lambda, BatchNormalization, Dropout
from keras import Sequential, Model
from keras.metrics import Precision, Recall
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.applications import MobileNetV3Large
from keras.applications import MobileNet
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet
from keras.applications import mobilenet
from keras.applications import resnet
from keras.applications import inception_v3
from keras.models import save_model

from sklearn.model_selection import train_test_split

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel(Model):
    def __init__(self, attribute, epochs, model_name, in_shape, batch_size, callbacks, final_layer_neurons,
                 pretrain_lr, finetune_lr,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        super().__init__()
        self.attribute = attribute
        self.model_name = model_name.lower()
        self.in_shape = in_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.pretrain_lr = pretrain_lr
        self.finetune_lr = finetune_lr
        self.callbacks = callbacks
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_pretrain, self.X_finetune, self.y_pretrain, self.y_finetune = train_test_split(
            X_train, y_train, test_size=0.01, random_state = 2
        )
        self.X_test = X_test
        self.y_test = y_test
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 1,      # Binary category = 1
            'neckline': 4,
            'fabric': 6,
            'fit': 3,
            'category' : 15
        }

        del X_train
        del y_train

        self.cat_num = self.cat_nums[self.attribute]
        self.activation ='softmax'
        self.loss = 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below

    def instantiate_model(self):
        input = Input(self.in_shape)
        #preprocess = Lambda(mobilenet.preprocess_input)

        if self.model_name == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = inception_v3.preprocess_input
        elif self.model_name == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = resnet.preprocess_input
        elif self.model_name == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = mobilenet.preprocess_input
        elif self.model_name == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = efficientnet.preprocess_input
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers
        outputs = base_model(input)
        outputs = BatchNormalization(name='BatchNormalization')(outputs)
        outputs = GlobalAveragePooling2D()(outputs)
        outputs = Dropout(0.5)(outputs)
        outputs = Dense(units=self.final_layer_neurons, activation='relu')(outputs)
        outputs = LeakyReLU(alpha=0.1)(outputs)
        outputs = Dropout(0.25)(outputs)
        outputs = Dense(units=self.cat_num, activation=self.activation)(outputs)
        self.model = Model(inputs = [input], outputs = [outputs])
        self.model.save('')
        print(self.loss) # check loss func
        self.model.compile(loss=self.loss, optimizer=Adam(learning_rate=self.pretrain_lr),
                      metrics=['accuracy'])
        return self.model

    def train(self):
            self.datagen = ImageDataGenerator(preprocessing_function=self.preprocess_input)
            self.X_pretrain, self.X_val, self.y_pretrain, self.y_val = train_test_split(
                self.X_pretrain, self.y_pretrain, test_size=0.2
            )
            self.pretrain_generator = self.datagen.flow(
            self.X_pretrain, self.y_pretrain,
            batch_size=16)
            self.pretrain_val = self.datagen.flow(
            self.X_val, self.y_val,
            batch_size=16)
            # Train the model on the images and labels
            self.model.history = self.model.fit(self.pretrain_generator,validation_data = self.pretrain_val,
            epochs=self.epochs, batch_size=self.batch_size, verbose=1,
            callbacks=self.callbacks)
            del self.pretrain_generator
            del self.pretrain_val
            return self.model.history

    def finetune(self,
                 metrics=['accuracy']):
        self.datagen = ImageDataGenerator(preprocessing_function=self.preprocess_input)
        self.X_finetune, self.X_fval, self.y_finetune, self.y_fval = train_test_split(
                self.X_finetune, self.y_finetune, test_size=0.2)
        self.finetune_generator = self.datagen.flow( 
        self.X_finetune, self.y_finetune,                                
        batch_size=16)

        self.X_fval = self.datagen.flow(
        self.X_fval, self.y_fval,
        batch_size=16)

        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(self.finetune_lr),                  # Very low learning rate
            loss=self.loss,
            metrics=metrics)
        self.model.fit(self.finetune_generator,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=self.callbacks)
        
        del self.finetune_generator
        return self.model, self.model.history

    def evaluate(self):
        self.test_generator = self.datagen.flow( 
        self.X_test, self.y_test,                                
        batch_size=16)
        res = self.model.evaluate(self.test_generator, verbose=1)
        del self.test_generator
        return res

    def predict(self, X):
        return self.model.predict(self.preprocess_input(X))


    def save(self, filepath):
        return save_model(self.model,
                                   filepath=filepath)
        
    def summary(self):
        return self.model.summary()


In [7]:
mobilenet_category = AttrModel(attribute='category',
                      model_name='mobilenet',
                      in_shape=(224,224,3),
                      batch_size= 16,
                      epochs=100,
                      pretrain_lr = .0001,
                      finetune_lr = .00001,
                      callbacks = EarlyStopping(patience=5, restore_best_weights=True),
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

categorical_crossentropy


In [8]:
mobilenet_category.train()

Epoch 1/100
496/496 [==============================] - 25s 30ms/step - loss: 2.0471 - accuracy: 0.3530 - val_loss: 1.1988 - val_accuracy: 0.6507
Epoch 2/100
496/496 [==============================] - 14s 28ms/step - loss: 1.2575 - accuracy: 0.5889 - val_loss: 0.8677 - val_accuracy: 0.7077
Epoch 3/100
496/496 [==============================] - 14s 29ms/step - loss: 1.0602 - accuracy: 0.6295 - val_loss: 0.7536 - val_accuracy: 0.7293
Epoch 4/100
496/496 [==============================] - 17s 34ms/step - loss: 0.9463 - accuracy: 0.6536 - val_loss: 0.6952 - val_accuracy: 0.7445
Epoch 5/100
496/496 [==============================] - 14s 29ms/step - loss: 0.8942 - accuracy: 0.6773 - val_loss: 0.6586 - val_accuracy: 0.7545
Epoch 6/100
496/496 [==============================] - 14s 28ms/step - loss: 0.8219 - accuracy: 0.7030 - val_loss: 0.6333 - val_accuracy: 0.7646
Epoch 7/100
496/496 [==============================] - 14s 28ms/step - loss: 0.7984 - accuracy: 0.7101 - val_loss: 0.6162 - val_ac

In [9]:
model=mobilenet_category.evaluate()

157/157 [==============================] - 4s 24ms/step - loss: 0.4796 - accuracy: 0.8216


In [26]:
mobilenet_category.save('/content/drive/MyDrive/Colab_Notebooks/model')

In [19]:
mobilenet_category.finetune()

Epoch 1/50
151/151 [==============================] - ETA: 0s - loss: 2.8225 - accuracy: 0.0940

151/151 [==============================] - 23s 70ms/step - loss: 2.8225 - accuracy: 0.0940
Epoch 2/50
151/151 [==============================] - ETA: 0s - loss: 2.5938 - accuracy: 0.1681

151/151 [==============================] - 10s 66ms/step - loss: 2.5938 - accuracy: 0.1681
Epoch 3/50
151/151 [==============================] - ETA: 0s - loss: 2.3210 - accuracy: 0.2683

151/151 [==============================] - 10s 68ms/step - loss: 2.3210 - accuracy: 0.2683
Epoch 4/50
151/151 [==============================] - ETA: 0s - loss: 2.1250 - accuracy: 0.3261

151/151 [==============================] - 10s 68ms/step - loss: 2.1250 - accuracy: 0.3261
Epoch 5/50
151/151 [==============================] - ETA: 0s - loss: 1.9438 - accuracy: 0.3873

151/151 [==============================] - 10s 68ms/step - loss: 1.9438 - accuracy: 0.3873
Epoch 6/50
 98/151 [==================>...........] - ETA: 3s - loss: 1.7868 - accuracy: 0.4503

KeyboardInterrupt: ignored

In [ ]:
mobilenet_category.evaluate()

# Model - VGG19 model

In [20]:
from keras.applications import VGG19

def load_model():
    
    # $CHALLENGIFY_BEGIN
    
    model = VGG19(weights="imagenet", include_top=False, input_shape=(224,224,3))
    
    # $CHALLENGIFY_END
    
    return model

In [21]:
model = load_model()
model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [22]:
def set_nontrainable_layers(model):
    
    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model

In [23]:
from keras.backend import dropout
from keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(1024, activation='relu')
    dropout_layer = layers.Dropout(0.5)
    dense_layer2 = layers.Dense(1024, activation='relu')
    prediction_layer = layers.Dense(15, activation='softmax')
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        dropout_layer,
        dense_layer2,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

In [24]:
model = add_last_layers(model)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              25691136  
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_4 (Dense)             (None, 15)                15375     
                                                                 
Total params: 46,780,495
Trainable params: 26,756,111
No

In [25]:
from keras import optimizers
from keras.metrics import Precision, Recall

def build_model():
    # $CHALLENGIFY_BEGIN    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
                  #metrics=[Precision(), Recall()])
    return model
    # $CHALLENGIFY_END

In [26]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_5 (Dense)             (None, 1024)              25691136  
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_7 (Dense)             (None, 15)                15375     
                                                                 
Total params: 46,780,495
Trainable params: 26,756,111


In [ ]:
X_train = X_train/256

In [31]:
from keras.callbacks import EarlyStopping 

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history1 = model.fit(X_train, y_train,
               epochs=10, 
               batch_size = 32,
               callbacks=[es], 
               validation_split=0.2)

Epoch 1/10
242/242 [==============================] - 65s 252ms/step - loss: 7.0790 - accuracy: 0.4690 - val_loss: 24.2604 - val_accuracy: 0.0000e+00
Epoch 2/10
 19/242 [=>............................] - ETA: 41s - loss: 2.4437 - accuracy: 0.4507

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_test, y_test)

76/76 [==============================] - 15s 202ms/step - loss: 2.3621 - accuracy: 0.2410


[2.362133264541626, 0.24097885191440582]

In [ ]:
model.save('test_model.tf')

Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7efde937a4c0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7efde937a4c0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
